

---

# RSA encryption
RSA is an asymmetric key algorithm where there is an encryption and decryption key. Since it is slow to implemenet, it is used to send the symmetric keys instead of encrypting the data.


Let's go through some Maths basics first to refresh what we learned in school.

# Greatest Common Divisor

**Euclid's algorithm** for determining the greatest common divisor (GCD).

a is larger than b, divide a by b, divide quotient by remainder, repeat till remainder is 0


In [1]:
#@title Greatest Common Divisor
'''
Euclid's algorithm for determining the greatest common divisor
Use iteration to make it faster for larger integers
a is larger than b, divide a by b, divide quotient by remainder, repeat till remainder is 0
'''
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

In [2]:
gcd(5,10)

5

# Relatively prime or coprimes
Two numbers a and b are relatively primes iff $gcd(a,b) = 1$ which means that they only share $1$ as their common factor.

For example, $14$ and $15$ are relatively primes

# Congurent numbers

> $a \equiv b \pmod{m}$ ---> $a$ is congruent to $b$ modulo $m$

which implies $m$ divides $a-b$

> $a - b = m * k$

> $b = remainder(a,m)$

# Multliplicative inverse

x and a are multiplicate inverse of each other mod m if

> $a * x ≡ 1 \pmod{m}$, where $x\in\{1, .. , m-1\}$

which means $m$ divides $a * x - 1$

> $a*x - 1 = m * k$

> $a*x = m * k + 1$

> $1 = remainder(ax,m)$

*It can be shown that if $gcd(n,k) =1$, then $k$ has a multiplicative inverse mod n*

In [3]:
#@title Naive Modular multiplicative inverse
'''
Euclid's extended algorithm for finding the multiplicative inverse of two numbers
ax ≡ 1 (mod m), x lies in [1, .. , m-1]
'''
def modular_multiplicative_inverse(a, m):
    a = a % m;
    for x in range(1, m) :
        if ((a * x) % m == 1) :
            return x
    return 1

In [4]:
#@title Modular Multiplicative Inverse Function
def mod_inv(a, m):
    """
    Computes the modular multiplicative inverse of a modulo m using the Extended Euclidean Algorithm.

    Parameters:
        a (int): The number to find the inverse of.
        m (int): The modulus.

    Returns:
        int: The modular multiplicative inverse of a mod m, or None if no inverse exists.
    """
    def extended_gcd(a, b):
        """Helper function to implement the Extended Euclidean Algorithm."""
        if b == 0:
            return a, 1, 0
        gcd, x1, y1 = extended_gcd(b, a % b)
        x = y1
        y = x1 - (a // b) * y1
        return gcd, x, y

    gcd, x, _ = extended_gcd(a, m)
    if gcd != 1:
        # Modular inverse doesn't exist if gcd(a, m) != 1
        return None
    else:
        # Ensure the result is positive
        return x % m

# Example Usage
a = 75  #@param {type:"number"}
m = 29  #@param {type:"number"}

inverse = mod_inv(a, m)
if inverse is not None:
    print(f"The modular multiplicative inverse of {a} mod {m} is {inverse}.")
else:
    print(f"No modular multiplicative inverse exists for {a} mod {m}.")

The modular multiplicative inverse of 75 mod 29 is 12.


# Prime number

In order to check if a number N is prime, we start dividing it with numbers from 2 onwards. Since

$N = \sqrt{N} * \sqrt{N} $

the factor can't be greater than $\sqrt{N}$

So we only have to check numbers till then.

In [5]:
#@title Naive is prime implementation
'''
Tests to see if a number is prime.
'''
def is_prime(num):
    if num == 2:
        return True
    if num < 2 or num % 2 == 0:
        return False
    for n in range(3, int(num**0.5)+2, 2): # you can speed it up a bit more with similar tricks ...
        if num % n == 0:
            return False
    return True

In [6]:
# example
is_prime(6320312)

False

If you need to check whether a long number is a primer number the previous approach won't work, you'll need a probabilistic primality test. There are different packages that include those kind of test, one of the simplest one is sympy

In [7]:
from sympy import isprime

# Check primality

number = 63203128476040426868745407319982284176275751373812593808244917063767803882168756818789652917269943419 #@param {type:"number"}

if isprime(number):
    print(f"{number} is prime.")
else:
    print(f"{number} is not prime.")

63203128476040426868745407319982284176275751373812593808244917063767803882168756818789652917269943419 is prime.


In [8]:
#@title Generate Prime greater than a given number
from sympy import nextprime

number = 67004178 #@param {type:"number"}
number = nextprime(number)

if isprime(number):
    print(f"{number} is prime.")
else:
    print(f"{number} is not prime.")

67004207 is prime.


# RSA - Rivest–Shamir–Adleman
Published in 1977 in MIT [pdf](https://people.csail.mit.edu/rivest/Rsapaper.pdf)

# Generation of public and private keys

1.   Generate two prime numbers $p$ and $q$

2.   Define $N = p * q$ called a *semiprime* (product of two primes)

3.   Define $ \phi =  (p-1) * (q-1)$ which is Euler Totient function (see Euler's theorem)

4.   Generate encryption key $e$ such that $gcd(e, \phi) = 1 $ where e lies between $1$ to $\phi -1 $

    i.e. e is relatively prime to $\phi$

   **Public key**: $(e, N)$ visible to everyone

5.   Generate decryption key $d$ such that $d$ is multiplicate inverse of $e$ module $\phi$

    $d*e ≡ 1 \pmod{\phi}$

    **Private key**: $(d, N)$ visible only to sender and your receiver.


In [9]:
#@title RSA Keypair Generator
from sympy import isprime

def generate_keypair(p, q):
    if not (isprime(p) and isprime(q)):
        raise ValueError('Both numbers must be prime.')
    elif p == q:
        raise ValueError('p and q cannot be equal')
    #n = pq
    n = p * q

    #Phi is the totient of n
    phi = (p-1) * (q-1)

    #Choose an integer e such that e and phi(n) are coprime or relatively prime
    e = 5 # most common value is 65537

    #Use Euclid's Algorithm to verify that e and phi(n) are coprime
    if gcd(e, phi) != 1:
        raise ValueError("e and phi(n) are not coprime!") # You need to generate new primes

    #Use Extended Euclid's Algorithm to generate the private key de ≡ 1 (mod phi)
    d = mod_inv(e, phi)

    #Return public and private keypair
    #Public key is (e, n) and private key is (d, n)
    return ((e, n), (d, n))

In [11]:
from sympy import randprime

min_digits = 100 #@param {type:"number"}
a= 10**min_digits
b= 10**(min_digits+1)-1

p = randprime(a,b)
q = randprime(a,b)

print("Prime p:", p)
print("Prime q:", q)

print("Generating your public/private keypairs now . . .")
public, private = generate_keypair(p, q)
(e,n)= public
print("Modulus N:", n)
print("Public key : ", public)
print("Private key: ", private)

Prime p: 26326488540592080939948888067597039299235561151460658345259970097213787157800692789733154566541871717
Prime q: 18792068060526795555032127831541289340738058730527809621308458936455836975671959216258100408367139667
Generating your public/private keypairs now . . .
Modulus N: 494729164449485134863336435783876565969394175401747774713189440964610869401510954867717469059619907527501416336968683131361287469484684391735326056860524830914686154692240233071345497661072659736098239
Public key :  (5, 494729164449485134863336435783876565969394175401747774713189440964610869401510954867717469059619907527501416336968683131361287469484684391735326056860524830914686154692240233071345497661072659736098239)
Private key:  (395783331559588107890669148627101252775515340321398219770551552771688695521208763894173975247695925985906287788679845309104217256277084601409364939897645491477005696818092879678954793335854147861669485, 4947291644494851348633364357838765659693941754017477747131894409646108694015

# Encryption step


>   $C = M^e \pmod{N}$



In [14]:
def encrypt(key,n, plaintext):
    #Convert each letter in the plaintext to numbers based on the character using rem(m^e,n) i.e. m^e mod n
    cipher = [pow(ord(char), key,n) for char in plaintext]
    #Return the array of bytes
    return cipher

In [17]:

message = input("Enter a message to encrypt with your public key:")
n = int(input("Enter the RSA modulus:"))
key = int(input("Enter the public key exponent:"))

encrypted_msg = encrypt(key,n, message)
print("Your encrypted message is: ")
print(encrypted_msg)

Enter a message to encrypt with your public key:La estoy liando parda
Enter the RSA modulus:4045560166591560551274253345695887446796704517189811150402904998224164794919626022001792997265314408530586357807185736296454261264323625410838246763254703871369947574392985711944957597720819334581184659
Enter the public key exponent:5
Your encrypted message is: 
[2535525376, 8587340257, 33554432, 10510100501, 20113571875, 21003416576, 16850581551, 25937424601, 33554432, 14693280768, 12762815625, 8587340257, 16105100000, 10000000000, 16850581551, 33554432, 17623416832, 8587340257, 19254145824, 10000000000, 8587340257]


# Decryption step


>   $M' = C^d \pmod{N}$


In [12]:
def decrypt(key,n, ciphertext):
    #Generate the plaintext based on the ciphertext and key using rem(m'^d,n)
    plain = [chr(pow(char, key,n)) for char in ciphertext]
    #Return the array of bytes as a string
    return ''.join(plain)

In [13]:

# Input string
input_string = input("Enter the encrypted message:")

# Convert the string to a list
encrypted_msg = list(map(int, input_string.strip("[]").split(", ")))

n = int(input("Enter the RSA modulus:"))
key = int(input("Enter the private key exponent:"))

print("Your message is:")
print(decrypt(key,n, encrypted_msg))

Enter the encrypted message:[15386239549, 10510100501, 16105100000, 20113571875, 8587340257, 13382255776, 10510100501, 33554432, 10510100501, 16105100000, 9509900499, 19254145824, 12762815625, 17623416832, 21003416576, 8587340257, 10000000000, 16850581551]
Enter the RSA modulus:494729164449485134863336435783876565969394175401747774713189440964610869401510954867717469059619907527501416336968683131361287469484684391735326056860524830914686154692240233071345497661072659736098239
Enter the private key exponent:395783331559588107890669148627101252775515340321398219770551552771688695521208763894173975247695925985906287788679845309104217256277084601409364939897645491477005696818092879678954793335854147861669485
Your message is:
mensaje encriptado
